In [1]:
import os

In [2]:
class Circle:
    def __init__(self, radius):
        self._radius = radius  # Using a single underscore to indicate a protected variable

    def calculate_area(self):
        return 3.14 * self._radius * self._radius

In [3]:
# Creating an instance of the class
circle_instance = Circle(radius=5)

In [4]:
# Calling the method without @property
area = circle_instance.calculate_area()

In [5]:
print(f"Area without @property: {area}")

Area without @property: 78.5


In [23]:
class CircleWithProperty:
    def __init__(self, radius):
        self._radius = radius  # Using a single underscore to indicate a protected variable

    @property
    def area(self):
        return 3.14 * self._radius * self._radius
    
    
    def test(self):
        area=self.area["suny"]
        return area

In [24]:

# Creating an instance of the class
circle_instance_with_property = CircleWithProperty(radius=5)

In [8]:

# Accessing the method with @property like an attribute
area_with_property = circle_instance_with_property.area
print(f"Area with @property: {area_with_property}")


Area with @property: 78.5


In [16]:
circle_instance_with_property.area

78.5

In [25]:
circle_instance_with_property.test()

TypeError: 'float' object is not subscriptable

In [12]:
from ensure import ensure_annotations

In [13]:
@ensure_annotations
def product(x:int,y:int)->int:
    return x*y
    

In [14]:
product(10,"hi")

EnsureError: Argument y of type <class 'str'> to <function product at 0x00000266B3BEBEE0> does not match annotation type <class 'int'>